In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings(action='ignore')
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

In [3]:
train = pd.read_csv('/users/jjong/desktop/vscode/Dacon_src/jeju/train.csv')
test = pd.read_csv('/users/jjong/desktop/vscode/Dacon_src/jeju/test.csv')
trade = pd.read_csv('/users/jjong/desktop/vscode/Dacon_src/jeju/international_trade.csv')
sample = pd.read_csv('/users/jjong/desktop/vscode/Dacon_src/jeju/sample_submission.csv')

In [4]:
#시계열 특성을 학습에 반영하기 위해 timestamp를 월, 일, 시간으로 나눕니다
train['year'] = train['timestamp'].apply(lambda x : int(x[0:4]))
train['month'] = train['timestamp'].apply(lambda x : int(x[5:7]))
train['day'] = train['timestamp'].apply(lambda x : int(x[8:10]))

test['year'] = test['timestamp'].apply(lambda x : int(x[0:4]))
test['month'] = test['timestamp'].apply(lambda x : int(x[5:7]))
test['day'] = test['timestamp'].apply(lambda x : int(x[8:10]))

trade['year'] = trade['기간'].apply(lambda x : int(x[0:4]))
trade['month'] = trade['기간'].apply(lambda x : int(x[5:7]))

In [5]:
def english(data, col):
    data[col] = data[col].apply(lambda x: 'TG' if '감귤' in x else x)
    data[col] = data[col].apply(lambda x: 'BC' if '브로콜리' in x else x)
    data[col] = data[col].apply(lambda x: 'RD' if '무' in x else x)
    data[col] = data[col].apply(lambda x: 'CR' if '당근' in x else x)
    data[col] = data[col].apply(lambda x: 'CB' if '양배추' in x else x)
    
    return data
english(trade, '품목명')

,기간,품목명,수출 중량,수출 금액,수입 중량,수입 금액,무역수지,year,month
0,2019-01,토마토(신선한 것이나 냉장한 것으로 한정한다),356571,990,0,0,990,2019,1
1,2019-01,양파,821330,222,4003206,1118,-896,2019,1
2,2019-01,쪽파,60,1,93405,128,-127,2019,1
3,2019-01,BC,160,1,638913,563,-562,2019,1
4,2019-01,CB,0,0,7580,38,-38,2019,1
...,...,...,...,...,...,...,...,...,...
1269,2023-02,포포(papaw)[파파야(papaya)],0,0,23830,71,-71,2023,2
1270,2023-02,사과,135165,351,0,0,351,2023,2
1271,2023-02,배,2206012,5411,1,0,5411,2023,2
1272,2023-02,신 체리[프루너스 체라서스(Prunus cerasus)],5,0,0,0,0,2023,2


In [6]:
trade = trade.rename(columns={'품목명':'item'})

In [7]:
filtered = trade[(trade['item'] == 'TG') | (trade['item'] == 'BC') | (trade['item'] == 'RD') | (trade['item'] == 'CR') | (trade['item'] == 'CB')]
# filtered.drop('기간', axis=1, inplace=True)
filtered

,기간,item,수출 중량,수출 금액,수입 중량,수입 금액,무역수지,year,month
3,2019-01,BC,160,1,638913,563,-562,2019,1
4,2019-01,CB,0,0,7580,38,-38,2019,1
5,2019-01,CB,184650,94,395802,90,4,2019,1
8,2019-01,CR,23150,22,7466150,2955,-2934,2019,1
12,2019-01,RD,2627,23,94529,464,-441,2019,1
...,...,...,...,...,...,...,...,...,...
1250,2023-02,CB,13188,13,377456,104,-91,2023,2
1253,2023-02,CR,22510,20,9260020,3758,-3737,2023,2
1254,2023-02,RD,4000,4,2,0,4,2023,2
1258,2023-02,RD,1319,14,104566,454,-440,2023,2


In [8]:
filtered = pd.get_dummies(filtered.drop('기간', axis=1))
filtered['기간'] = filtered['year'].astype(str) + '-' + filtered['month'].astype(str)
filtered.drop(['year', 'month'], axis=1, inplace=True)
filtered

,수출 중량,수출 금액,수입 중량,수입 금액,무역수지,item_BC,item_CB,item_CR,item_RD,item_TG,기간
3,160,1,638913,563,-562,True,False,False,False,False,2019-1
4,0,0,7580,38,-38,False,True,False,False,False,2019-1
5,184650,94,395802,90,4,False,True,False,False,False,2019-1
8,23150,22,7466150,2955,-2934,False,False,True,False,False,2019-1
12,2627,23,94529,464,-441,False,False,False,True,False,2019-1
...,...,...,...,...,...,...,...,...,...,...,...
1250,13188,13,377456,104,-91,False,True,False,False,False,2023-2
1253,22510,20,9260020,3758,-3737,False,False,True,False,False,2023-2
1254,4000,4,2,0,4,False,False,False,True,False,2023-2
1258,1319,14,104566,454,-440,False,False,False,True,False,2023-2


In [9]:
fil_group = filtered.groupby('기간').mean()

In [10]:
train.drop(['ID', 'timestamp', 'day'], axis=1, inplace=True)
train['기간'] = train['year'].astype(str) + '-' + train['month'].astype(str)
train

,item,corporation,location,supply(kg),price(원/kg),year,month,기간
0,TG,A,J,0.0,0.0,2019,1,2019-1
1,TG,A,J,0.0,0.0,2019,1,2019-1
2,TG,A,J,60601.0,1728.0,2019,1,2019-1
3,TG,A,J,25000.0,1408.0,2019,1,2019-1
4,TG,A,J,32352.0,1250.0,2019,1,2019-1
...,...,...,...,...,...,...,...,...
59392,RD,F,J,452440.0,468.0,2023,2,2023-2
59393,RD,F,J,421980.0,531.0,2023,2,2023-2
59394,RD,F,J,382980.0,574.0,2023,3,2023-3
59395,RD,F,J,477220.0,523.0,2023,3,2023-3


In [11]:
train = pd.get_dummies(train.drop('기간', axis=1))
train['timestamp'] = train['year'].astype(str) + '-' + train['month'].astype(str)
train.groupby('timestamp').mean()
train

,supply(kg),price(원/kg),year,month,item_BC,item_CB,item_CR,item_RD,item_TG,corporation_A,corporation_B,corporation_C,corporation_D,corporation_E,corporation_F,location_J,location_S,timestamp
0,0.0,0.0,2019,1,False,False,False,False,True,True,False,False,False,False,False,True,False,2019-1
1,0.0,0.0,2019,1,False,False,False,False,True,True,False,False,False,False,False,True,False,2019-1
2,60601.0,1728.0,2019,1,False,False,False,False,True,True,False,False,False,False,False,True,False,2019-1
3,25000.0,1408.0,2019,1,False,False,False,False,True,True,False,False,False,False,False,True,False,2019-1
4,32352.0,1250.0,2019,1,False,False,False,False,True,True,False,False,False,False,False,True,False,2019-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59392,452440.0,468.0,2023,2,False,False,False,True,False,False,False,False,False,False,True,True,False,2023-2
59393,421980.0,531.0,2023,2,False,False,False,True,False,False,False,False,False,False,True,True,False,2023-2
59394,382980.0,574.0,2023,3,False,False,False,True,False,False,False,False,False,False,True,True,False,2023-3
59395,477220.0,523.0,2023,3,False,False,False,True,False,False,False,False,False,False,True,True,False,2023-3


In [12]:
train = train.rename(columns={'timestamp':'기간'})

In [13]:
filtered

,수출 중량,수출 금액,수입 중량,수입 금액,무역수지,item_BC,item_CB,item_CR,item_RD,item_TG,기간
3,160,1,638913,563,-562,True,False,False,False,False,2019-1
4,0,0,7580,38,-38,False,True,False,False,False,2019-1
5,184650,94,395802,90,4,False,True,False,False,False,2019-1
8,23150,22,7466150,2955,-2934,False,False,True,False,False,2019-1
12,2627,23,94529,464,-441,False,False,False,True,False,2019-1
...,...,...,...,...,...,...,...,...,...,...,...
1250,13188,13,377456,104,-91,False,True,False,False,False,2023-2
1253,22510,20,9260020,3758,-3737,False,False,True,False,False,2023-2
1254,4000,4,2,0,4,False,False,False,True,False,2023-2
1258,1319,14,104566,454,-440,False,False,False,True,False,2023-2


In [14]:
tmp = pd.merge(train, filtered, on='기간', how='inner')
tmp

,supply(kg),price(원/kg),year,month,item_BC_x,item_CB_x,item_CR_x,item_RD_x,item_TG_x,corporation_A,...,수출 중량,수출 금액,수입 중량,수입 금액,무역수지,item_BC_y,item_CB_y,item_CR_y,item_RD_y,item_TG_y
0,0.0,0.0,2019,1,False,False,False,False,True,True,...,160,1,638913,563,-562,True,False,False,False,False
1,0.0,0.0,2019,1,False,False,False,False,True,True,...,0,0,7580,38,-38,False,True,False,False,False
2,0.0,0.0,2019,1,False,False,False,False,True,True,...,184650,94,395802,90,4,False,True,False,False,False
3,0.0,0.0,2019,1,False,False,False,False,True,True,...,23150,22,7466150,2955,-2934,False,False,True,False,False
4,0.0,0.0,2019,1,False,False,False,False,True,True,...,2627,23,94529,464,-441,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362695,421980.0,531.0,2023,2,False,False,False,True,False,False,...,13188,13,377456,104,-91,False,True,False,False,False
362696,421980.0,531.0,2023,2,False,False,False,True,False,False,...,22510,20,9260020,3758,-3737,False,False,True,False,False
362697,421980.0,531.0,2023,2,False,False,False,True,False,False,...,4000,4,2,0,4,False,False,False,True,False
362698,421980.0,531.0,2023,2,False,False,False,True,False,False,...,1319,14,104566,454,-440,False,False,False,True,False


In [15]:
tmp = tmp.groupby('기간').mean()

In [16]:
test.drop(['ID', 'timestamp'], axis=1, inplace=True)
test = pd.get_dummies(test)
test

,year,month,day,item_BC,item_CB,item_CR,item_RD,item_TG,corporation_A,corporation_B,corporation_C,corporation_D,corporation_E,corporation_F,location_J,location_S
0,2023,3,4,False,False,False,False,True,True,False,False,False,False,False,True,False
1,2023,3,5,False,False,False,False,True,True,False,False,False,False,False,True,False
2,2023,3,6,False,False,False,False,True,True,False,False,False,False,False,True,False
3,2023,3,7,False,False,False,False,True,True,False,False,False,False,False,True,False
4,2023,3,8,False,False,False,False,True,True,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1087,2023,3,27,False,False,False,True,False,False,False,False,False,False,True,True,False
1088,2023,3,28,False,False,False,True,False,False,False,False,False,False,True,True,False
1089,2023,3,29,False,False,False,True,False,False,False,False,False,False,True,True,False
1090,2023,3,30,False,False,False,True,False,False,False,False,False,False,True,True,False


In [17]:
y = train['price(원/kg)']
X = train.drop(['price(원/kg)', 'supply(kg)', '기간'], axis=1)

In [18]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_sc = scaler.fit_transform(X)

In [19]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Conv2D, MaxPooling2D, BatchNormalization, Activation, GlobalMaxPooling2D

inputs = Input(shape=(X_sc.shape[0], X_sc.shape[1], 1)) # 흑백채널 하나

x = Conv2D(128, (3, 3), padding='same')(inputs) # 필터 수, 커널사이즈
x = MaxPooling2D((2,2), strides=2)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Conv2D(128, (3, 3), padding='same')(x) 
x = MaxPooling2D((2,2), strides=2)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Conv2D(256, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = GlobalMaxPooling2D()(x) # GlobalAverageFooling2D를 요즘 거의 많이 씀 , 채널을 압축시켜줌
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
outputs = Dense(10, activation='softmax')(x)

model2 = Model(inputs=inputs, outputs=outputs)
model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model2.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 59397, 15, 1)]    0         
                                                                 
 conv2d (Conv2D)             (None, 59397, 15, 128)    1280      
                                                                 
 max_pooling2d (MaxPooling2  (None, 29698, 7, 128)     0         
 D)                                                              
                                                                 
 batch_normalization (Batch  (None, 29698, 7, 128)     512       
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 29698, 7, 128)     0         
                                                                 
 conv2d_1 (Conv2D)           (None, 29698, 7, 128)     147584

In [20]:
from tensorflow.keras.callbacks import EarlyStopping

stopper = EarlyStopping(monitor='val_loss', patience=5, mode='min', restore_best_weights=True)
model2.fit(X_sc, y, epochs=100, batch_size=128, validation_split=0.2, callbacks=[stopper], verbose=2)

Epoch 1/100


ValueError: in user code:

    File "c:\Users\jjong\Desktop\vscode\CWNU_ICT\tensor\Lib\site-packages\keras\src\engine\training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\jjong\Desktop\vscode\CWNU_ICT\tensor\Lib\site-packages\keras\src\engine\training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\jjong\Desktop\vscode\CWNU_ICT\tensor\Lib\site-packages\keras\src\engine\training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\jjong\Desktop\vscode\CWNU_ICT\tensor\Lib\site-packages\keras\src\engine\training.py", line 1080, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\jjong\Desktop\vscode\CWNU_ICT\tensor\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\jjong\Desktop\vscode\CWNU_ICT